<a href="https://colab.research.google.com/github/harshithamadarapu/Team16_Hinglish-Auto-suggestions/blob/main/TinyBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load TinyBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = AutoModelForMaskedLM.from_pretrained("huawei-noah/TinyBERT_General_4L_312D").to(device)

# Print the device being used
print(f"Using device: {device}")

Using device: cuda


In [5]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your preprocessed datasets
train_df = pd.read_csv('/content/preprocessed_train.csv')
valid_df = pd.read_csv('/content/prerocessed_validation (1).csv')

# Check the columns and structure
print(train_df.head(), valid_df.head())

# Load TinyBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_4L_312D")
model = AutoModelForMaskedLM.from_pretrained("huawei-noah/TinyBERT_General_4L_312D").to(device)

# Create Dataset from the data
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["phrases"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_valid = valid_dataset.map(tokenize_function, batched=True)

# Data collator for masked language modeling (MLM)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./hinglish_tinybert_model",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=10,
    logging_dir="./logs",
    logging_steps=10,
)

# Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# Fine-tune the model
trainer.train()

                                             phrases
0                               film ka kya naam hai
1  namaste sada hua tomatoes score mahaan hai lek...
2  kya aapako lagata hai ki aapako film pasand aa...
3                        yah kis tarah kee philm hai
4                                film kab banee thee                                              phrases
0  snow may fall for parts of cny the next few da...
1  kaale megha kaale megha paani toh barsao megha...
2                            sallu bhai kaha ho thum
3     guessthesong aao kho jaaye sitaaron mein kahin
4  chidambaram ka kuch nahi ukhaad paayenge becua...


Map:   0%|          | 0/188525 [00:00<?, ? examples/s]

Map:   0%|          | 0/2064 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-8d45837e12ee>:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.070800,5.130867
2,1.964200,4.891701
